In [1]:
import requests
import json
import pandas as pd
import numpy as np
from pymongo import MongoClient
import math
from bs4 import BeautifulSoup
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
#import src.functions as fn

In [2]:
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

## Indice calidad de vida

En primer lugar, voy a basarme en el índice de calidad de vida para seleccionar una zona. 

El índice de calidad de vida se calcula en base otros índices: índice de poder adquisitivo, índice de seguridad, indice de sanidad, de costo de visa de tiempo de desplazamiento den tráfico, contaminación clima y relación precio/ingresos para propiedades.

Para obtener esta información he hecho scrapping de la web https://es.numbeo.com/calidad-de-vida/clasificaciones-por-pa%C3%ADs.

In [3]:
data=requests.get("https://es.numbeo.com/calidad-de-vida/clasificaciones-por-pa%C3%ADs").text
soup= BeautifulSoup(data, 'html.parser')

def procesaIndices(fila):
    m = fila.find_all("td")
    #print(m[2])
    return {
        "country":m[1].text.strip(),
        "calidad_vida":float((m[2].text).replace(',','.')),
        "poder_adquisitivo":float((m[3].text).replace(',','.')),
        "seguridad":float((m[4].text).replace(',','.')),
        "sanidad":float((m[5].text).replace(',','.')),
        "costo_vida":float((m[6].text).replace(',','.')),
        "relacion_precio_vs_ingresos":float((m[7].text).replace(',','.')),
        "tiempo_desplazamiento":float((m[8].text).replace(',','.')),
        "contaminación":float((m[9].text).replace(',','.')),
        "clima":float((m[10].text).replace(',','.'))
    }
    
indice = soup.find_all('table')[2]
indice_dict = [procesaIndices(fila) for fila in indice.find_all("tr")[1:]]


In [4]:
#Genero df:
df_indices=pd.DataFrame(indice_dict)

#Exporto en formato json y luego importo en MongoDB Compass:
df_indices.to_json("output/indices_calidad.json", orient="records")

El país que elegiré para poner la nueva sede, será aquel con mejor indice de calidad y además con un indice de clima superior a 90. Como se ve a continuación será Australia.

In [5]:
df_indices[(df_indices.clima>90)].head()

,country,calidad_vida,poder_adquisitivo,seguridad,sanidad,costo_vida,relacion_precio_vs_ingresos,tiempo_desplazamiento,contaminación,clima
3,Australia,186.21,107.31,58.64,77.38,73.54,7.52,34.73,23.46,92.70
7,Nueva Zelanda,181.02,92.66,59.07,73.81,72.53,8.52,31.10,23.40,95.46
14,España,169.82,72.03,68.04,78.88,53.77,9.37,29.10,39.99,94.19
18,Portugal,162.91,49.43,70.37,71.88,49.52,12.70,30.00,30.89,97.31
25,Francia,153.95,80.36,53.21,79.99,74.14,13.04,34.76,43.56,90.25


## Importo de MongoDB Compass:
Con una query me traeré de la colección 'companies' todas las empresas que tenga una ganancia distinta de 0$.

In [6]:
q1={"total_money_raised": {"$ne": "$0"}}
companies=list(db["companies"].find(q1,{"name":1,"founded_year":1,"total_money_raised":1, "offices":1,"category_code":1}))

In [7]:
df = pd.DataFrame(companies)

Voy a tratar el df para desagregar la columna offices, dejando cada oficina en una línea distina y luego desdoblando las columnas que contienen la información de cada oficina.

In [8]:
#Desagrego las oficinas, para que por compañía cada oficina aparezca en una línea independiente:
df = df.explode('offices')
#display(df.head())

#Desagrego la columna office para obtener los datos en columnas de cada oficina:
df_offices = df[["offices"]].apply(lambda x: x.offices, result_type="expand", axis=1)
#display(df_offices.head())

#Unifico los dos df anteriores para tener un único df completo con toda la información
clean_data = pd.concat([df,df_offices], axis=1)
display(clean_data.head())

,_id,name,category_code,founded_year,total_money_raised,offices,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,52cdef7c4bab8bd675297d8f,Omnidrive,network_hosting,2005.0,$800k,"{'description': '', 'address1': 'Suite 200', '...",,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN
1,52cdef7c4bab8bd675297d91,Geni,web,2006.0,$16.5M,"{'description': 'Headquarters', 'address1': '9...",Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,2005.0,$39.8M,"{'description': '', 'address1': '710 - 2nd Ave...",,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,2005.0,$39.8M,"{'description': '', 'address1': '270 Lafayette...",,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
3,52cdef7c4bab8bd675297d96,Gizmoz,web,2003.0,$18.1M,"{'description': None, 'address1': None, 'addre...",None,None,None,None,Menlo Park,CA,USA,37.484130,-122.169472


Elimino la columna _id, ya que se generará un nuevo id cuando cargue la nueva colección, y también elimino office que ya está toda la información desagregada y no nos sirve más.

In [9]:
clean_data = clean_data.drop(columns=["_id","offices"])
clean_data.head()

,name,category_code,founded_year,total_money_raised,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,Omnidrive,network_hosting,2005.0,$800k,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN
1,Geni,web,2006.0,$16.5M,Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064
2,Wetpaint,web,2005.0,$39.8M,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
2,Wetpaint,web,2005.0,$39.8M,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
3,Gizmoz,web,2003.0,$18.1M,None,None,None,None,Menlo Park,CA,USA,37.484130,-122.169472


Con la siguiente función, daré el formato GeoJson, necesario para poder generar los indexes en MongoDB Compass. En aquellos casos en los que latitud o longitud sean valores NaN, se indicará None, para evitar posibles conflicos en la carga de la colección.

In [10]:
def asGeoJSON(lat,lng):
    try:
        lat = float(lat)
        lng = float(lng)
        if not math.isnan(lat) and not math.isnan(lng):
            return {
                "type":"Point",
                "coordinates":[lng,lat]
            }
    except Exception:
        print("Invalid data")
        return None
        

clean_data["location"] = clean_data[["latitude","longitude"]].apply(lambda x:asGeoJSON(x.latitude,x.longitude), axis=1)
clean_data[["latitude","longitude","location"]].head()

,latitude,longitude,location
0,NaN,NaN,None
1,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,..."
2,47.603122,-122.333253,"{'type': 'Point', 'coordinates': [-122.333253,..."
2,40.723731,-73.996431,"{'type': 'Point', 'coordinates': [-73.9964312,..."
3,37.484130,-122.169472,"{'type': 'Point', 'coordinates': [-122.169472,..."


Añado una columna que tipifique si la empresa tiene 10 o menos años('young') o más de 10 años ('old').

In [11]:
def oldYoung(row):
    if row['founded_year']>=2007:
        return 'young'
    else:
        return 'old'

clean_data['old_young']= clean_data.apply(lambda row: oldYoung(row), axis=1)

clean_data[['name', 'founded_year','old_young']].head(10)

,name,founded_year,old_young
0,Omnidrive,2005.0,old
1,Geni,2006.0,old
2,Wetpaint,2005.0,old
2,Wetpaint,2005.0,old
3,Gizmoz,2003.0,old
4,Lala,NaN,old
5,StumbleUpon,2002.0,old
5,StumbleUpon,2002.0,old
6,Slacker,2006.0,old
7,Scribd,2007.0,young


Exporto a json y luego lo importo a MongoDB Compass.

In [12]:
clean_data.to_json("output/cleaned_offices.json", orient="records")

Lo importo en Mongo DB Compass: $ mongoimport --db companies --collection offices --jsonArray --drop cleaned_offices.json

Una vez importado, creo un geospartial index: Indexes > Create Index > (write any indexname) > Select fieldname:"location" and 2dsphere

Genero query para traeme los aeropuertos que hay en Australia

## Consultas a GooglePlace: starbucks, pubs, parques...

Request a la API de google consultando los starbucks en sydney.

In [20]:
import os
from dotenv import load_dotenv
load_dotenv('src/.env')

def requestGooglePlace(place):
    token = os.getenv("API_GOOGLE_KEY")
    #print(token)
    if not token:
        raise ValueError("Necesitas un API_GOOGLE_KEY")
    
    baseUrl = "https://maps.googleapis.com/"
    endpoint="maps/api/place/textsearch/json"
    url = baseUrl+endpoint
    print(f"Requesting data from {url}")
    params = {
        "query":f"{place}",
        "key": f"{token}"
    }
    res = requests.get(url,params=params)
    if res.status_code != 200:
        print(res.text)
        raise ValueError("Bad Response")
    return res.json()

#Prueba: data_Starbucks=requestGooglePlace("Starbucks+in+Sydney")

## Request de Starbucks en Melbourne

Elijo Melbourne como ciudad para establecer la oficina, ya que hay más empresas jóvenes que en Sydney.

In [23]:
#data_Starbucks_Melbourn=requestGooglePlace("Starbucks+in+Melbourn+Australia")

Extraigo la key "results" y convierto a dataframe para trabajar con la columna "geometry"

In [24]:
results_mel=data_Starbucks_Melbourn['results']
df_starbucks_melbourn = pd.DataFrame(results_mel)

#Guardo el response de la request lanzada:
json_starbucks_melbourn=json.dumps(results_mel)
writeFile =open('starbucks_melbourn.json', 'w')
writeFile.write(json_starbucks_melbourn)
writeFile.close()

In [25]:
#Desagrego la columna geometry.location para obtener la latitud y longitud en columnas de cada Starbucks:
df_loc_mel_cafe = df_starbucks_melbourn[["geometry"]].apply(lambda x: x.geometry, result_type="expand", axis=1)

df_loc_mel_cafe2 = df_loc_mel_cafe[["location"]].apply(lambda x: x.location, result_type="expand", axis=1)
#display(df_loc_syd_cafe2.head())

#Unifico los dos df anteriores para tener un único df completo con toda la información
clean_starbucks_melbourn = pd.concat([df_starbucks_melbourn,df_loc_mel_cafe2], axis=1)
display(clean_starbucks_melbourn.head())

,formatted_address,geometry,icon,id,name,opening_hours,photos,place_id,plus_code,price_level,rating,reference,types,user_ratings_total,lat,lng
0,"295 Swanston St, Melbourne VIC 3000, Australia","{'location': {'lat': -37.8113086, 'lng': 144.9...",https://maps.gstatic.com/mapfiles/place_api/ic...,428668de1bda3a762d61c901dfdaf78a5ae6a559,Starbucks,{'open_now': True},"[{'height': 1920, 'html_attributions': ['<a hr...",ChIJgfh7WspC1moRCKdd0o5WEy8,"{'compound_code': '5XQ7+FP Melbourne, Victoria...",2,3.9,ChIJgfh7WspC1moRCKdd0o5WEy8,"[cafe, food, point_of_interest, store, establi...",948,-37.811309,144.964310
1,"Tivoli Arcade, 235-251 Bourke St, Melbourne VI...","{'location': {'lat': -37.8134938, 'lng': 144.9...",https://maps.gstatic.com/mapfiles/place_api/ic...,1ca754c4e5b67f2463b63dc33797c46b31915538,Starbucks,{'open_now': True},"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJ8TrsBcpC1moRJLQvmKZ1_6k,"{'compound_code': '5XP8+JG Melbourne, Victoria...",2,3.9,ChIJ8TrsBcpC1moRJLQvmKZ1_6k,"[cafe, food, point_of_interest, store, establi...",839,-37.813494,144.966352
2,"Spencer Outlet Centre, 201 Spencer St, Melbour...","{'location': {'lat': -37.8161662, 'lng': 144.9...",https://maps.gstatic.com/mapfiles/place_api/ic...,1251ab8f72fb0d4edaaa27a920bf4d1a4e1503d9,Starbucks,{'open_now': True},"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJYx8bFVBd1moRMTl1DA1h5mk,"{'compound_code': '5XM2+GR Melbourne, Victoria...",2,3.7,ChIJYx8bFVBd1moRMTl1DA1h5mk,"[cafe, food, point_of_interest, store, establi...",573,-37.816166,144.952079
3,"2 Elizabeth St, Melbourne VIC 3004, Australia","{'location': {'lat': -37.8178938, 'lng': 144.9...",https://maps.gstatic.com/mapfiles/place_api/ic...,173a0b592f6846db854f2f13d37cdeed18b991bf,Starbucks,{'open_now': True},"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJt44AHw1D1moRipc46Fz7uzA,"{'compound_code': '5XJ8+R2 Melbourne, Victoria...",2,3.6,ChIJt44AHw1D1moRipc46Fz7uzA,"[cafe, food, point_of_interest, store, establi...",164,-37.817894,144.965043
4,"Melbourne Central, 151/377-391 Swanston St, Me...","{'location': {'lat': -37.8098483, 'lng': 144.9...",https://maps.gstatic.com/mapfiles/place_api/ic...,ea41651ed0a99037b576e209ce73a783581d4b4f,Starbucks,{'open_now': True},"[{'height': 1600, 'html_attributions': ['<a hr...",ChIJXV05DMtC1moRfLFTfqwfW-4,"{'compound_code': '5XR7+3G Melbourne, Victoria...",2,4.0,ChIJXV05DMtC1moRfLFTfqwfW-4,"[cafe, food, point_of_interest, store, establi...",649,-37.809848,144.963781


Añado columna con formato geoquery:

In [26]:
clean_starbucks_melbourn["location"] = clean_starbucks_melbourn[["lat","lng"]].apply(lambda x:asGeoJSON(x.lat,x.lng), axis=1)
new_clean_starbucks_melbourn = clean_starbucks_melbourn[["name","lat","lng","location"]]
new_clean_starbucks_melbourn.head()

,name,lat,lng,location
0,Starbucks,-37.811309,144.964310,"{'type': 'Point', 'coordinates': [144.96431, -..."
1,Starbucks,-37.813494,144.966352,"{'type': 'Point', 'coordinates': [144.9663518,..."
2,Starbucks,-37.816166,144.952079,"{'type': 'Point', 'coordinates': [144.9520791,..."
3,Starbucks,-37.817894,144.965043,"{'type': 'Point', 'coordinates': [144.965043, ..."
4,Starbucks,-37.809848,144.963781,"{'type': 'Point', 'coordinates': [144.9637812,..."


In [27]:
new_clean_starbucks_melbourn.to_json("output/new_clean_starbucks_melbourn.json", orient="records")

## Request Pubs en Melbourne

In [29]:
#data_pubs=requestGooglePlace("pubs+in+Melbourne+Australia")

Requesting data from https://maps.googleapis.com/maps/api/place/textsearch/json


In [30]:
results_pubs_melbourn=data_pubs['results']
df_pubs_melbourn = pd.DataFrame(results_pubs_melbourn)
json_pubs_melbourn=json.dumps(results_pubs_melbourn)

writeFile =open('output/pubs_melbourn.json', 'w')
writeFile.write(json_pubs_melbourn)
writeFile.close()

In [31]:
#Desagrego la columna geometry.location para obtener la latitud y longitud en columnas de cada Starbucks:
df_loc_mel_pubs = df_pubs_melbourn[["geometry"]].apply(lambda x: x.geometry, result_type="expand", axis=1)

df_loc_mel_pubs2 = df_loc_mel_pubs[["location"]].apply(lambda x: x.location, result_type="expand", axis=1)
#display(df_loc_mel_pubs2.head())

#Unifico los dos df anteriores para tener un único df completo con toda la información
clean_pubs_melbourn = pd.concat([df_pubs_melbourn,df_loc_mel_pubs2], axis=1)
display(clean_pubs_melbourn.head())

,formatted_address,geometry,icon,id,name,opening_hours,photos,place_id,plus_code,price_level,rating,reference,types,user_ratings_total,lat,lng
0,"146 Flinders St, Melbourne VIC 3000, Australia","{'location': {'lat': -37.8165307, 'lng': 144.9...",https://maps.gstatic.com/mapfiles/place_api/ic...,b60ed44212ce40d7e77395f17b018c03f4e97c5c,The Duke of Wellington Pub & Rooftop Bar,{'open_now': True},"[{'height': 1365, 'html_attributions': ['<a hr...",ChIJ9YePsLdC1moRhmhASJxsSqk,"{'compound_code': '5XMC+92 Melbourne, Victoria...",2,4.0,ChIJ9YePsLdC1moRhmhASJxsSqk,"[bar, restaurant, food, point_of_interest, est...",1760,-37.816531,144.970054
1,"127 Russell St, Melbourne VIC 3000, Australia","{'location': {'lat': -37.813709, 'lng': 144.96...",https://maps.gstatic.com/mapfiles/place_api/ic...,86a2eea528177996e0cf33a3fa8ea23fa4c927e4,The Crafty Squire,{'open_now': True},"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJazndwslC1moRIfHbO-8g0ug,"{'compound_code': '5XP9+G8 Melbourne, Victoria...",2,4.2,ChIJazndwslC1moRIfHbO-8g0ug,"[bar, restaurant, food, point_of_interest, est...",2476,-37.813709,144.968264
2,"427 Little Collins St, Melbourne VIC 3000, Aus...","{'location': {'lat': -37.8162205, 'lng': 144.9...",https://maps.gstatic.com/mapfiles/place_api/ic...,725ad1f0351293cf67c58f216fc5aeb5de20e1a4,The Irish Times Pub,{'open_now': True},"[{'height': 495, 'html_attributions': ['<a hre...",ChIJidRmV0td1moR9NIteGY9FmM,"{'compound_code': '5XM6+G5 Melbourne, Victoria...",2,4.3,ChIJidRmV0td1moR9NIteGY9FmM,"[bar, restaurant, food, point_of_interest, est...",1105,-37.816221,144.960498
3,"101 Flinders Ln, Melbourne VIC 3000, Australia","{'location': {'lat': -37.81542, 'lng': 144.971...",https://maps.gstatic.com/mapfiles/place_api/ic...,2c4f243aa70358d4e317ae2455c201b12b84cdb9,Garden State Hotel,{'open_now': True},"[{'height': 1500, 'html_attributions': ['<a hr...",ChIJ90Gk4bdC1moRHBHcy_R_0UY,"{'compound_code': '5XMC+RF Melbourne, Victoria...",2,4.2,ChIJ90Gk4bdC1moRHBHcy_R_0UY,"[bar, restaurant, food, point_of_interest, est...",2654,-37.815420,144.971164
4,"263 William St, Melbourne VIC 3000, Australia","{'location': {'lat': -37.812874, 'lng': 144.95...",https://maps.gstatic.com/mapfiles/place_api/ic...,7c75d754b4301d6bc52404a50a6d68321d14d118,The Metropolitan Hotel,{'open_now': True},"[{'height': 1333, 'html_attributions': ['<a hr...",ChIJx_g3zEtd1moRP7mUwZodVko,"{'compound_code': '5XP4+VM Melbourne, Victoria...",2,4.0,ChIJx_g3zEtd1moRP7mUwZodVko,"[bar, restaurant, food, point_of_interest, est...",616,-37.812874,144.956739


In [32]:
clean_pubs_melbourn["location"] = clean_pubs_melbourn[["lat","lng"]].apply(lambda x:asGeoJSON(x.lat,x.lng), axis=1)
new_clean_pubs_melbourn = clean_pubs_melbourn[["name","lat","lng","location"]]
new_clean_pubs_melbourn.head()

,name,lat,lng,location
0,The Duke of Wellington Pub & Rooftop Bar,-37.816531,144.970054,"{'type': 'Point', 'coordinates': [144.9700544,..."
1,The Crafty Squire,-37.813709,144.968264,"{'type': 'Point', 'coordinates': [144.968264, ..."
2,The Irish Times Pub,-37.816221,144.960498,"{'type': 'Point', 'coordinates': [144.9604985,..."
3,Garden State Hotel,-37.815420,144.971164,"{'type': 'Point', 'coordinates': [144.971164, ..."
4,The Metropolitan Hotel,-37.812874,144.956739,"{'type': 'Point', 'coordinates': [144.9567388,..."


In [33]:
new_clean_pubs_melbourn.to_json("output/new_clean_pubs_melbourn.json", orient="records")

## Request Schools en Melbourne

In [37]:
#data_schools = requestGooglePlace("schools+in+Melbourne+Australia")

In [38]:
results_schools_melbourne = data_schools['results']
df_schools_melbourne = pd.DataFrame(results_schools_melbourne)
json_schools_melbourne = json.dumps(results_schools_melbourne)

writeFile =open('output/schools_melbourne.json', 'w')
writeFile.write(json_schools_melbourne)
writeFile.close()

In [39]:
#Desagrego la columna geometry.location para obtener la latitud y longitud en columnas de cada Starbucks:
df_loc_mel_schools = df_schools_melbourne[["geometry"]].apply(lambda x: x.geometry, result_type="expand", axis=1)

df_loc_mel_schools2 = df_loc_mel_schools[["location"]].apply(lambda x: x.location, result_type="expand", axis=1)
#display(df_loc_mel_schools2.head())

#Unifico los dos df anteriores para tener un único df completo con toda la información
clean_schools_melbourne = pd.concat([df_schools_melbourne,df_loc_mel_schools2], axis=1)
display(clean_schools_melbourne.head())

,formatted_address,geometry,icon,id,name,photos,place_id,plus_code,rating,reference,types,user_ratings_total,opening_hours,lat,lng
0,"355 St Kilda Rd, Melbourne VIC 3004, Australia","{'location': {'lat': -37.8340741, 'lng': 144.9...",https://maps.gstatic.com/mapfiles/place_api/ic...,1a3f75ac50f3a8876d2eddc55ba16472b939a661,Melbourne Grammar School,"[{'height': 468, 'html_attributions': ['<a hre...",ChIJDQAavqdC1moR96s2QiX6Xj4,"{'compound_code': '5X8F+9X Melbourne, Victoria...",4.4,ChIJDQAavqdC1moR96s2QiX6Xj4,"[school, point_of_interest, establishment]",50,NaN,-37.834074,144.974942
1,"217 Glen Eira Rd, St Kilda East VIC 3183, Aust...","{'location': {'lat': -37.8764432, 'lng': 145.0...",https://maps.gstatic.com/mapfiles/place_api/ic...,25fd198a091ca978f9adae45001f7a1d4df570a8,Caulfield Grammar School - Caulfield Campus,"[{'height': 399, 'html_attributions': ['<a hre...",ChIJnX6j4ABq1moRA6yR3lrv85Q,"{'compound_code': '42F3+CC St Kilda East, Vict...",4.0,ChIJnX6j4ABq1moRA6yR3lrv85Q,"[secondary_school, school, point_of_interest, ...",34,{'open_now': False},-37.876443,145.003562
2,"577 St Kilda Rd, Melbourne VIC 3004, Australia","{'location': {'lat': -37.8490504, 'lng': 144.9...",https://maps.gstatic.com/mapfiles/place_api/ic...,4339cf562b9c0a5a857ac2efde51cea0995b5b36,Wesley College,"[{'height': 3840, 'html_attributions': ['<a hr...",ChIJh60GMhZo1moR3LX34gmbjA8,"{'compound_code': '5X2J+9W Melbourne, Victoria...",3.7,ChIJh60GMhZo1moR3LX34gmbjA8,"[school, primary_school, secondary_school, poi...",58,{'open_now': False},-37.849050,144.982299
3,"1 Morrison St, Hawthorn VIC 3122, Australia","{'location': {'lat': -37.8342314, 'lng': 145.0...",https://maps.gstatic.com/mapfiles/place_api/ic...,b9d35f796f19ce232a2918542d46078904f0a7a3,Scotch College,"[{'height': 2969, 'html_attributions': ['<a hr...",ChIJeVMRxj9C1moRqlqZkPbYJBs,"{'compound_code': '528H+8M Hawthorn, Victoria,...",3.6,ChIJeVMRxj9C1moRqlqZkPbYJBs,"[secondary_school, university, school, point_o...",130,NaN,-37.834231,145.029162
4,"88 Nicholson St, Fitzroy VIC 3065, Australia","{'location': {'lat': -37.8036637, 'lng': 144.9...",https://maps.gstatic.com/mapfiles/place_api/ic...,66ceb0993d42f5cf0bd1e9cc65bf0b151c4a0929,Academy of Mary Immaculate,"[{'height': 1365, 'html_attributions': ['<a hr...",ChIJNb9YRtpC1moR0Uq-xLdEoAo,"{'compound_code': '5XWF+GP Fitzroy, Victoria, ...",3.6,ChIJNb9YRtpC1moR0Uq-xLdEoAo,"[secondary_school, school, point_of_interest, ...",17,{'open_now': False},-37.803664,144.974264


In [40]:
clean_schools_melbourne["location"] = clean_schools_melbourne[["lat","lng"]].apply(lambda x:asGeoJSON(x.lat,x.lng), axis=1)
new_clean_schools_melbourne = clean_schools_melbourne[["name","lat","lng","location"]]
new_clean_schools_melbourne.head()

,name,lat,lng,location
0,Melbourne Grammar School,-37.834074,144.974942,"{'type': 'Point', 'coordinates': [144.9749422,..."
1,Caulfield Grammar School - Caulfield Campus,-37.876443,145.003562,"{'type': 'Point', 'coordinates': [145.0035616,..."
2,Wesley College,-37.849050,144.982299,"{'type': 'Point', 'coordinates': [144.9822987,..."
3,Scotch College,-37.834231,145.029162,"{'type': 'Point', 'coordinates': [145.0291624,..."
4,Academy of Mary Immaculate,-37.803664,144.974264,"{'type': 'Point', 'coordinates': [144.9742637,..."


In [41]:
new_clean_schools_melbourne.to_json("output/new_clean_schools_melbourn.json", orient="records")

## Request Parks in Melbourne

In [44]:
#data_parks = requestGooglePlace("schools+in+Melbourne+Australia")

Requesting data from https://maps.googleapis.com/maps/api/place/textsearch/json


In [45]:
results_parks_melbourne =data_parks['results']
df_parks_melbourne = pd.DataFrame(results_parks_melbourne)
json_parks_melbourne = json.dumps(results_parks_melbourne)

writeFile =open('output/parks_melbourne.json', 'w')
writeFile.write(json_parks_melbourne)
writeFile.close()

In [46]:
#Desagrego la columna geometry.location para obtener la latitud y longitud en columnas de cada Starbucks:
df_loc_mel_parks = df_parks_melbourne[["geometry"]].apply(lambda x: x.geometry, result_type="expand", axis=1)

df_loc_mel_parks2 = df_loc_mel_parks[["location"]].apply(lambda x: x.location, result_type="expand", axis=1)
#display(df_loc_mel_schools2.head())

#Unifico los dos df anteriores para tener un único df completo con toda la información
clean_parks_melbourne = pd.concat([df_parks_melbourne,df_loc_mel_parks2], axis=1)
display(clean_parks_melbourne.head())

,formatted_address,geometry,icon,id,name,photos,place_id,plus_code,rating,reference,types,user_ratings_total,opening_hours,lat,lng
0,"355 St Kilda Rd, Melbourne VIC 3004, Australia","{'location': {'lat': -37.8340741, 'lng': 144.9...",https://maps.gstatic.com/mapfiles/place_api/ic...,1a3f75ac50f3a8876d2eddc55ba16472b939a661,Melbourne Grammar School,"[{'height': 468, 'html_attributions': ['<a hre...",ChIJDQAavqdC1moR96s2QiX6Xj4,"{'compound_code': '5X8F+9X Melbourne, Victoria...",4.4,ChIJDQAavqdC1moR96s2QiX6Xj4,"[school, point_of_interest, establishment]",50,NaN,-37.834074,144.974942
1,"217 Glen Eira Rd, St Kilda East VIC 3183, Aust...","{'location': {'lat': -37.8764432, 'lng': 145.0...",https://maps.gstatic.com/mapfiles/place_api/ic...,25fd198a091ca978f9adae45001f7a1d4df570a8,Caulfield Grammar School - Caulfield Campus,"[{'height': 399, 'html_attributions': ['<a hre...",ChIJnX6j4ABq1moRA6yR3lrv85Q,"{'compound_code': '42F3+CC St Kilda East, Vict...",4.0,ChIJnX6j4ABq1moRA6yR3lrv85Q,"[secondary_school, school, point_of_interest, ...",34,{'open_now': False},-37.876443,145.003562
2,"577 St Kilda Rd, Melbourne VIC 3004, Australia","{'location': {'lat': -37.8490504, 'lng': 144.9...",https://maps.gstatic.com/mapfiles/place_api/ic...,4339cf562b9c0a5a857ac2efde51cea0995b5b36,Wesley College,"[{'height': 3840, 'html_attributions': ['<a hr...",ChIJh60GMhZo1moR3LX34gmbjA8,"{'compound_code': '5X2J+9W Melbourne, Victoria...",3.7,ChIJh60GMhZo1moR3LX34gmbjA8,"[school, primary_school, secondary_school, poi...",58,{'open_now': False},-37.849050,144.982299
3,"1 Morrison St, Hawthorn VIC 3122, Australia","{'location': {'lat': -37.8342314, 'lng': 145.0...",https://maps.gstatic.com/mapfiles/place_api/ic...,b9d35f796f19ce232a2918542d46078904f0a7a3,Scotch College,"[{'height': 2969, 'html_attributions': ['<a hr...",ChIJeVMRxj9C1moRqlqZkPbYJBs,"{'compound_code': '528H+8M Hawthorn, Victoria,...",3.6,ChIJeVMRxj9C1moRqlqZkPbYJBs,"[secondary_school, university, school, point_o...",130,NaN,-37.834231,145.029162
4,"88 Nicholson St, Fitzroy VIC 3065, Australia","{'location': {'lat': -37.8036637, 'lng': 144.9...",https://maps.gstatic.com/mapfiles/place_api/ic...,66ceb0993d42f5cf0bd1e9cc65bf0b151c4a0929,Academy of Mary Immaculate,"[{'height': 1365, 'html_attributions': ['<a hr...",ChIJNb9YRtpC1moR0Uq-xLdEoAo,"{'compound_code': '5XWF+GP Fitzroy, Victoria, ...",3.6,ChIJNb9YRtpC1moR0Uq-xLdEoAo,"[secondary_school, school, point_of_interest, ...",17,{'open_now': False},-37.803664,144.974264


In [47]:
clean_parks_melbourne["location"] = clean_parks_melbourne[["lat","lng"]].apply(lambda x:asGeoJSON(x.lat,x.lng), axis=1)
new_clean_parks_melbourne = clean_parks_melbourne[["name","lat","lng","location"]]
new_clean_parks_melbourne.head()

,name,lat,lng,location
0,Melbourne Grammar School,-37.834074,144.974942,"{'type': 'Point', 'coordinates': [144.9749422,..."
1,Caulfield Grammar School - Caulfield Campus,-37.876443,145.003562,"{'type': 'Point', 'coordinates': [145.0035616,..."
2,Wesley College,-37.849050,144.982299,"{'type': 'Point', 'coordinates': [144.9822987,..."
3,Scotch College,-37.834231,145.029162,"{'type': 'Point', 'coordinates': [145.0291624,..."
4,Academy of Mary Immaculate,-37.803664,144.974264,"{'type': 'Point', 'coordinates': [144.9742637,..."


In [48]:
new_clean_parks_melbourne.to_json("output/new_clean_parks_melbourn.json", orient="records")

## Request Veggie Restaurants in Melbourne

In [49]:
#data_veggies = requestGooglePlace("veggie+restaurants+in+Melbourne+Australia")

Requesting data from https://maps.googleapis.com/maps/api/place/textsearch/json


In [50]:
results_veggies_melbourne=data_veggies['results']
df_veggies_melbourne = pd.DataFrame(results_veggies_melbourne)
json_veggies_melbourne = json.dumps(results_veggies_melbourne)

writeFile =open('output/veggies_melbourne.json', 'w')
writeFile.write(json_veggies_melbourne)
writeFile.close()

In [51]:
#Desagrego la columna geometry.location para obtener la latitud y longitud en columnas de cada Starbucks:
df_loc_mel_veggies = df_veggies_melbourne[["geometry"]].apply(lambda x: x.geometry, result_type="expand", axis=1)

df_loc_mel_veggies2 = df_loc_mel_veggies[["location"]].apply(lambda x: x.location, result_type="expand", axis=1)
#display(df_loc_mel_schools2.head())

#Unifico los dos df anteriores para tener un único df completo con toda la información
clean_veggies_melbourne = pd.concat([df_veggies_melbourne,df_loc_mel_veggies2], axis=1)
display(clean_veggies_melbourne.head())

,formatted_address,geometry,icon,id,name,opening_hours,photos,place_id,plus_code,price_level,rating,reference,types,user_ratings_total,lat,lng
0,"201-203 Faraday St, Carlton VIC 3053, Australia","{'location': {'lat': -37.798995, 'lng': 144.96...",https://maps.gstatic.com/mapfiles/place_api/ic...,40d47607f85daf1269bf7ef29312b484f185bda3,Shakahari Vegetarian Restaurant,{'open_now': True},"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJUTD1ldZC1moRIJ1lbzkoFqY,"{'compound_code': '6X29+C4 Carlton, Victoria, ...",2.0,4.4,ChIJUTD1ldZC1moRIJ1lbzkoFqY,"[restaurant, food, point_of_interest, establis...",264,-37.798995,144.967807
1,"139 Swanston St, Melbourne VIC 3000, Australia","{'location': {'lat': -37.8144077, 'lng': 144.9...",https://maps.gstatic.com/mapfiles/place_api/ic...,973f29a88d7754a08b1d4e79f99f7cbbdffa9c9f,Gopals Pure Vegetarian,{'open_now': True},"[{'height': 2988, 'html_attributions': ['<a hr...",ChIJa68S77VC1moRTdJLPEIk4Ok,"{'compound_code': '5XP8+68 Melbourne, Victoria...",1.0,4.5,ChIJa68S77VC1moRTdJLPEIk4Ok,"[restaurant, food, point_of_interest, establis...",856,-37.814408,144.965762
2,"380 Brunswick St, Fitzroy VIC 3065, Australia","{'location': {'lat': -37.7958399, 'lng': 144.9...",https://maps.gstatic.com/mapfiles/place_api/ic...,5407859a2926f9998c0f66fdd2390392760acd66,Vegie Bar,{'open_now': True},"[{'height': 2112, 'html_attributions': ['<a hr...",ChIJ2VtFFSFD1moRSdG9BsaTtow,"{'compound_code': '6X3H+MM Fitzroy, Victoria, ...",2.0,4.3,ChIJ2VtFFSFD1moRSdG9BsaTtow,"[bar, restaurant, food, point_of_interest, est...",2159,-37.795840,144.979135
3,"175 Brunswick St, Fitzroy VIC 3065, Australia","{'location': {'lat': -37.8022152, 'lng': 144.9...",https://maps.gstatic.com/mapfiles/place_api/ic...,885b8aecc60e0d65e82053c225174f8fd934d646,Smith & Daughters,{'open_now': True},"[{'height': 1536, 'html_attributions': ['<a hr...",ChIJvRkxNdlC1moRH3O0se8KS3s,"{'compound_code': '5XXH+42 Fitzroy, Victoria, ...",2.0,4.4,ChIJvRkxNdlC1moRH3O0se8KS3s,"[restaurant, food, point_of_interest, establis...",1078,-37.802215,144.977612
4,The Walk Arcade The Causeway opposite Causeway...,"{'location': {'lat': -37.8144479, 'lng': 144.9...",https://maps.gstatic.com/mapfiles/place_api/ic...,32181a20d4bdb951828c5876d22e1f3c0d4207b2,Crossways Vegetarian and Vegan Restaurant,{'open_now': True},"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJJQvjwrVC1moR5Bno6k5ep-w,"{'compound_code': '5XP7+6R Melbourne, Victoria...",1.0,4.7,ChIJJQvjwrVC1moR5Bno6k5ep-w,"[restaurant, food, point_of_interest, establis...",1110,-37.814448,144.964567


In [52]:
clean_veggies_melbourne["location"] = clean_veggies_melbourne[["lat","lng"]].apply(lambda x:asGeoJSON(x.lat,x.lng), axis=1)
new_clean_veggies_melbourne = clean_veggies_melbourne[["name","lat","lng","location"]]
new_clean_veggies_melbourne.head()

,name,lat,lng,location
0,Shakahari Vegetarian Restaurant,-37.798995,144.967807,"{'type': 'Point', 'coordinates': [144.967807, ..."
1,Gopals Pure Vegetarian,-37.814408,144.965762,"{'type': 'Point', 'coordinates': [144.9657624,..."
2,Vegie Bar,-37.795840,144.979135,"{'type': 'Point', 'coordinates': [144.9791354,..."
3,Smith & Daughters,-37.802215,144.977612,"{'type': 'Point', 'coordinates': [144.9776118,..."
4,Crossways Vegetarian and Vegan Restaurant,-37.814448,144.964567,"{'type': 'Point', 'coordinates': [144.9645668,..."


In [53]:
new_clean_veggies_melbourne.to_json("output/new_clean_veggies_melbourn.json", orient="records")

## Mejor ubicación en Melbourne:

Compruebo cuál de las oficinas de Melbourne tiene mejor ubicación.

In [56]:
query_young= {"$and":[{"location": {"$ne":None}}, {"city":{"$eq": "Melbourne"}}]}
melbourne_office=list(db["offices"].find(query_young,{"_id":0,"name":1,"old_young":1,"total_money_raised":1, "offices":1,"category_code":1, "location":1, "latitude":1, "longitude":1}))
df_melbourne_office=pd.DataFrame(melbourne_office)

df_melbourne_office.to_json("output/melbourne_offices.json", orient="records")

df_melbourne_office

,name,category_code,total_money_raised,latitude,longitude,location,old_young
0,Google,search,$555M,-37.879234,145.073608,"{'type': 'Point', 'coordinates': [145.073608, ...",old
1,Hitwise,web,$4.2M,-37.814251,144.963169,"{'type': 'Point', 'coordinates': [144.963169, ...",old
2,Direct Hit,search,$26M,28.085421,-80.647501,"{'type': 'Point', 'coordinates': [-80.6475007,...",old
3,Coremetrics,advertising,$91M,-37.814541,144.970500,"{'type': 'Point', 'coordinates': [144.9705, -3...",old
4,Aconex,software,$85M,-37.816116,144.971542,"{'type': 'Point', 'coordinates': [144.9715423,...",old
5,Travel Distribution Systems,enterprise,$1.1M,-37.819691,145.205688,"{'type': 'Point', 'coordinates': [145.205688, ...",old
6,Internet Marketing Academy Australia,other,$100k,-37.839941,144.976425,"{'type': 'Point', 'coordinates': [144.976425, ...",young


Lista de localizaciones de las oficinas de Melbourne para cruzar con las colecciones creadas: starbucks, parques, colegios, pubs y restaurantes veggies y ver cuántos de cada uno hay a menos de 5km.

In [57]:
list_melbourne_offices=[]
for e in range(len(df_melbourne_office)):
    list_melbourne_offices.append(df_melbourne_office["location"][e])


Defino la función que contará el número de starbucks, parques, colegios, pubs y restaurantes veggies que están a menos de 5 km de cada oficina.

In [73]:
def countNearStarbucks(list_location , maxDistance):
    
    return list(db.starbucks_melbourne.find({
       "location": {
         "$near": {
           "$geometry": list_location,
           "$maxDistance": maxDistance
         }
       }
    }
    )
    )

maxDistance=1000
lista_starbucks=[]
num_starbucks=[]

for i in range(len(list_melbourne_offices)):
    coincidencias= countNearStarbucks(list_melbourne_offices[i], maxDistance)
    lista_starbucks.append(coincidencias)
    num_starbucks.append(len(coincidencias))

df_melbourne_office['num_starbucks_1km']=num_starbucks


In [74]:
def countNearPubs(list_location, maxDistance):
    
    return list(db.pubs_melbourne.find({
       "location": {
         "$near": {
           "$geometry": list_location,
           "$maxDistance": maxDistance
         }
       }
    }
    )
    )

maxDistance=2000
lista_pubs=[]
num_pubs=[]

for i in range(len(list_melbourne_offices)):
    coincidencias= countNearPubs(list_melbourne_offices[i], maxDistance)
    lista_pubs.append(coincidencias)
    num_pubs.append(len(coincidencias))

df_melbourne_office['num_pubs_2km']=num_pubs

In [75]:
def countNearParks(list_location, maxDistance):
    
    return list(db.parks_melbourne.find({
       "location": {
         "$near": {
           "$geometry": list_location,
           "$maxDistance": maxDistance
         }
       }
    }
    )
    )

maxDistance=1000
lista_parks=[]
num_parks=[]

for i in range(len(list_melbourne_offices)):
    coincidencias= countNearPubs(list_melbourne_offices[i], maxDistance)
    lista_parks.append(coincidencias)
    num_parks.append(len(coincidencias))

df_melbourne_office['num_parks_1km']=num_parks

In [76]:
def countNearSchools(list_location, maxDistance):
    
    return list(db.schools_melbourne.find({
       "location": {
         "$near": {
           "$geometry": list_location,
           "$maxDistance": maxDistance
         }
       }
    }
    )
    )

maxDistance=5000
lista_schools=[]
num_schools=[]

for i in range(len(list_melbourne_offices)):
    coincidencias= countNearPubs(list_melbourne_offices[i], maxDistance)
    lista_schools.append(coincidencias)
    num_schools.append(len(coincidencias))

df_melbourne_office['num_schools_5km']=num_schools

In [77]:
def countNearVeggies(list_location, maxDistance):
    
    return list(db.veggies_melbourne.find({
       "location": {
         "$near": {
           "$geometry": list_location,
           "$maxDistance": maxDistance
         }
       }
    }
    )
    )

maxDistance=1000
lista_veggies=[]
num_veggies=[]

for i in range(len(list_melbourne_offices)):
    coincidencias= countNearPubs(list_melbourne_offices[i], maxDistance)
    lista_veggies.append(coincidencias)
    num_veggies.append(len(coincidencias))

df_melbourne_office['num_veggies_1km']=num_veggies

In [122]:
df_melbourne_office['ranking']= df_melbourne_office['num_starbucks_1km']*0.3 + df_melbourne_office['num_pubs_2km']*0.2 + df_melbourne_office['num_parks_1km']*0.2 +df_melbourne_office['num_schools_5km']*0.15+df_melbourne_office['num_veggies_1km']*0.15
df_melbourne_office.sort_values(by=['ranking'], ascending = False)

,name,category_code,total_money_raised,latitude,longitude,location,old_young,num_starbucks,num_pubs,num_parks,num_schools,num_veggies,num_starbucks_1km,num_pubs_2km,num_parks_1km,num_schools_5km,num_veggies_1km,ranking
1,Hitwise,web,$4.2M,-37.814251,144.963169,"{'type': 'Point', 'coordinates': [144.963169, ...",old,7,18,15,20,18,7,18,15,20,15,13.95
3,Coremetrics,advertising,$91M,-37.814541,144.970500,"{'type': 'Point', 'coordinates': [144.9705, -3...",old,6,18,12,20,18,6,18,12,20,12,12.60
4,Aconex,software,$85M,-37.816116,144.971542,"{'type': 'Point', 'coordinates': [144.9715423,...",old,6,18,12,20,18,6,18,12,20,12,12.60
6,Internet Marketing Academy Australia,other,$100k,-37.839941,144.976425,"{'type': 'Point', 'coordinates': [144.976425, ...",young,0,1,0,20,1,0,1,0,20,0,3.20
0,Google,search,$555M,-37.879234,145.073608,"{'type': 'Point', 'coordinates': [145.073608, ...",old,0,0,0,0,0,0,0,0,0,0,0.00
2,Direct Hit,search,$26M,28.085421,-80.647501,"{'type': 'Point', 'coordinates': [-80.6475007,...",old,0,0,0,0,0,0,0,0,0,0,0.00
5,Travel Distribution Systems,enterprise,$1.1M,-37.819691,145.205688,"{'type': 'Point', 'coordinates': [145.205688, ...",old,0,0,0,0,0,0,0,0,0,0,0.00


## -------------------------------------------------------------------------------------------------------------
Según el raking de valoración hecho, la ubicación de nuestra oficina estará justo encima de la empresa Hitwise (latitude=-37.814251 y longitude=144.963169)
## -------------------------------------------------------------------------------------------------------------

## MongoDB:

Me traigo las oficinas que están en Australia para ver en un mapa su distribución:

In [84]:
q_off_aus={"$and": [{"country_code": {"$eq": "AUS"}}, {"location": {"$ne":None}}]}
aus = list(db["offices"].find(q_off_aus, {'_id':0,'name':1, 'total_money_raised':1,'category_code':1, 'founded_year':1, 'city':1, 'latitude':1, 'longitude':1, 'old_young':1}))
df_aus = pd.DataFrame(aus)
df_aus.head()

,name,category_code,founded_year,total_money_raised,city,latitude,longitude,old_young
0,Google,search,1998.0,$555M,Melbourne,-37.879234,145.073608,old
1,Google,search,1998.0,$555M,Sydney,-34.822723,138.612396,old
2,Hitwise,web,NaN,$4.2M,Melbourne,-37.814251,144.963169,old
3,2threads,games_video,2004.0,$300k,sydney,-33.884685,151.216427,old
4,99designs,design,2008.0,$35M,Collingwood,-37.802659,144.986855,young


## Mapa en folium:

Antes consultaré las coordenadas de Australia en geoCode con la siguiente función:

In [96]:
def geocode(address):
    data = requests.get(f"https://geocode.xyz/{address}?json=1").json()
    print(data)
    return {
        "type":"Point",
        "coordinates":[float(data["longt"]),float(data["latt"])]
    }

aus_geo = geocode("Australia")



{'longt': '0.00000', 'standard': {'addresst': {}, 'prov': 'AU', 'city': {}, 'countryname': 'Australia', 'postal': {}, 'confidence': '0.90'}, 'matches': None, 'alt': {}, 'error': {'description': '15. Your request did not produce any results.', 'code': '008'}, 'suggestion': {'region': 'AU', 'locate': {}}, 'latt': '0.00000'}


Primero, muestro el mapa de Australia con todas las oficinas que hay en el país y que tienen localización

In [98]:
start_lat = -25.2743988
start_lon = 133.7751312
aus_map = folium.Map(location=[start_lat, start_lon],tiles='cartodbpositron', zoom_start=4)

df_aus.apply(lambda row:folium.Marker(location=[row["latitude"], 
                                                row["longitude"]], popup=row["name"],icon=folium.Icon(color='blue', icon_color='white', icon='')).add_to(aus_map),axis=1)
aus_map

In [133]:
#Hago zoom a la ciudad de Melbourne:

start_lat = -37.8139992
start_lon = 144.9633179
melbourne_map = folium.Map(location=[start_lat, start_lon],tiles='cartodbpositron', zoom_start=10)


#Incluyo el aeropuerto:
q_air={"$and": [{"country": {"$eq":"Australia"}}, {"city":{"$eq":"Melbourne"}}]}
aus_airport = list(db['airports'].find(q_air,{"_id":0, "name":1,"city":1,"lat":1,"lon":1}))

df_airports = pd.DataFrame (aus_airport)
df_airports.head()


df_airports.apply(lambda row:folium.Marker(location=[row["lat"], 
                                                row["lon"]], popup=row["name"],icon=folium.Icon(color='black', icon_color='white', icon='plane')).add_to(melbourne_map),axis=1)

#Incluyo la ubicación seleccionada (latitude=-37.814251 y longitude=144.963169):
select_lon, select_lat = 144.963169, -37.814251
Marker([select_lat, select_lon], icon=folium.Icon(color='red')).add_to(melbourne_map)

#Incluyo los Starbucks, pubs, parques, colegios y restaurantes veggies que cumplen las distancias máximas establecidas:
pd.DataFrame(lista_starbucks[1]).apply(lambda row:folium.Marker(location=[row["lat"], 
                                                row["lng"]], popup=row["name"],icon=folium.Icon(color='darkgreen', icon_color='white',icon='S')).add_to(melbourne_map),axis=1)


pd.DataFrame(lista_pubs[1]).apply(lambda row:folium.Marker(location=[row["lat"], 
                                                row["lng"]], popup=row["name"],icon=folium.Icon(color='pink', icon_color='white', icon='glass')).add_to(melbourne_map),axis=1)

pd.DataFrame(lista_schools[1]).apply(lambda row:folium.Marker(location=[row["lat"], 
                                                row["lng"]], popup=row["name"],icon=folium.Icon(color='lightblue', icon_color='white',icon='S')).add_to(melbourne_map),axis=1)

pd.DataFrame(lista_parks[1]).apply(lambda row:folium.Marker(location=[row["lat"], 
                                                row["lng"]], popup=row["name"],icon=folium.Icon(color='lightgreen', icon_color='white',icon='S')).add_to(melbourne_map),axis=1)

pd.DataFrame(lista_veggies[1]).apply(lambda row:folium.Marker(location=[row["lat"], 
                                                row["lng"]], popup=row["name"],icon=folium.Icon(color='orange', icon_color='white',icon='S')).add_to(melbourne_map),axis=1)



melbourne_map
